# <center>Analysis of DART impact simulations</center>

## 0) Imports

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import h5py

%load_ext nb_black
%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 10)
pd.set_option("display.precision", 2)

<IPython.core.display.Javascript object>

## 1) Load simulation data:

In [2]:
file = "/scratch/rutz/DidymoonImpact/data/noporous/impact_noporous.0300.h5"

<IPython.core.display.Javascript object>

In [23]:
with h5py.File(file, "r") as hdf:
    keys = list(hdf.keys())

keys

['DIM_root_of_damage_tensile',
 'a',
 'a_grav',
 'activation_thresholds',
 'dddt',
 'ddeviatoric_stress_dt',
 'dedt',
 'deviatoric_stress',
 'drhodt',
 'e',
 'local_strain',
 'm',
 'material_type',
 'maximum_number_of_flaws',
 'number_of_activated_flaws',
 'number_of_interactions',
 'p',
 'rho',
 'sml',
 'sml_initial',
 'soundspeed',
 'time',
 'tree_depth',
 'v',
 'x']

<IPython.core.display.Javascript object>

In [28]:
with h5py.File(file, "r") as hdf:
    df = pd.DataFrame(
        {
            "x": hdf.get("x")[:, 0],
            "y": hdf.get("x")[:, 1],
            "z": hdf.get("x")[:, 2],
            "v_x": hdf.get("v")[:, 0],
            "v_y": hdf.get("v")[:, 1],
            "v_z": hdf.get("v")[:, 2],
            "m": hdf.get("m"),
            "mat": hdf.get("material_type"),
        }
    )
    df["v_abs"] = np.sqrt(df["v_x"] ** 2 + df["v_y"] ** 2 + df["v_z"] ** 2)

df

,x,y,z,v_x,v_y,v_z,m,mat,v_abs
0,1.52,-0.82,-2.00,12.75,-8.02,-6.62,12.78,0,16.46
1,1.16,-1.26,-2.07,9.79,-10.73,-6.34,12.78,0,15.85
2,0.61,-0.99,-2.74,8.56,-7.80,-14.19,12.78,0,18.32
3,-0.43,-1.10,-2.61,-1.07,-7.99,-15.16,12.78,0,17.17
4,1.72,-0.60,-1.93,13.58,-6.21,-2.98,12.91,0,15.22
...,...,...,...,...,...,...,...,...,...
110190,-19.49,-2.41,-0.08,-1.34,-0.15,-0.23,1404.09,1,1.36
110191,-19.56,-1.66,-0.33,-1.23,-0.11,-0.20,1404.09,1,1.25
110192,-19.61,-0.87,-0.39,-1.26,-0.09,-0.21,1404.09,1,1.28
110193,-19.63,-0.09,-0.26,-1.30,-0.04,-0.22,1404.09,1,1.32


<IPython.core.display.Javascript object>

## 2) Beta factor

###  2.0) Physical parameters

In [141]:
rho_didymoon = 2860.0
r_surface = 75
M = 4 / 3 * np.pi * r_surface ** 3 * rho_didymoon
G = 6.6743e-11

<IPython.core.display.Javascript object>

In [142]:
impactor_momentum = (df.loc[df["mat"] == 0]["m"] * 6000).sum()

<IPython.core.display.Javascript object>

### 2.1) Escape velocity

In [143]:
df["v_escape"] = np.sqrt(2 * G * M / (r_surface + df.z))

<IPython.core.display.Javascript object>

### 2.2) Conditions for particles to be ejecta

In [144]:
filt_beta = (df["z"] > 0) & (df["v_z"] > 0) & (df["v_abs"] > df["v_escape"])
df_beta = df[filt_beta]
print(f"Number of ejecta particles: {filt_beta.sum()}")

Number of ejecta particles: 2193


<IPython.core.display.Javascript object>

### 2.3) Compute beta factor

In [145]:
recoil_momentum = (df[filt_beta]["m"] * df[filt_beta]["v_z"]).sum()

<IPython.core.display.Javascript object>

In [146]:
beta = (impactor_momentum + recoil_momentum) / impactor_momentum
print(f"Beta factor: {beta:.2f}")

Beta factor: 2.41


<IPython.core.display.Javascript object>